## Practice notebook for project

### Usefil web resources
1. [sppinaler-neiromophic-classifier](https://github.com/alandiamond/spinnaker-neuromorphic-classifier/blob/master/Spinn5-LargeModel-LiveSpiking/src/python/Classifier_LiveSpikingInput.py)
2. [stdp-mnist](https://github.com/Hananel-Hazan/stdp-mnist/blob/master/code/train/snn_mnist.py)
3. [SNN_from_scratch](http://www.mjrlab.org/wp-content/uploads/2014/05/network_python_tutorial2013.pdf)  
4. [Nengo - pynn wrapper](http://compneuro.uwaterloo.ca/files/publications/bekolay.2014.pdf)
5. [Nengo - paper combining DNN and Nengo](https://arxiv.org/pdf/1805.11144.pdf)
6. [PyNN Official Documentation](http://neuralensemble.org/docs/PyNN/)